# Otimização de Alocação de Ordens de Serviço

## Introdução

Neste notebook vamos trabalhar com o IBM Decision Optimization para fazer a otimização da alocação das ordens de serviço entre diversas empresas provedoras de serviços. No caso que trabalharemos, teremos como premissas de que qualquer empresa de qualquer estado pode agir em ordens de serviço de outros estados, porém o preço é mais elevado devido ao deslocamento. Considerando que em alguns estados temos menos empresas prestadoras, e em outros empresas com menos técnicos para atuar, é importante conseguir escolher dentre aquelas que gerarão o menor custo possível.

Trabalharemos aqui com três datasets:

* Técnicos: o dataset de técnicos contém a lista de todas as empresas prestadoras, o estado das mesmas e a capacidade técnica (número de OSs que podem atuar por dia) em cada especialidade (Instalação, Manutenção, Assistência de Informática e Assistência de Telefonia);
* Preços: neste dataset temos a lista das empresas prestadoras e o preço que cada uma cobra para atuar em cada UF;
* OS: este dataset contém todas as ordens de serviço abertas, a UF de cada OS, a especialidade necessária e o número de dias que ela está aberta;

Vamos então começar nosso modelo!

## Carregando e visualizando os dados

Vamos inicialmente carregar nossos datasets. No IBM Watson Studio, clicando em uma célula de código em branco e indo no ícone no canto superior direito com um código binário, para adicionar dados, podemos facilmente inserir a conexão com data assets que temos no nosso projeto. Podemos inserí-los como um dataframe Pandas na célula abaixo:

In [1]:
## Insira aqui o dataframe para Tecnicos

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_c86a1d2d7061465cb304eb925d845ec7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='MKJjobhZ_4BD8V9vereARbfx7eOgP_MiJuy65SHW5v6P',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c86a1d2d7061465cb304eb925d845ec7.get_object(Bucket='workshoptim-donotdelete-pr-6u07kwfqd4dp4h',Key='Tecnicos.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Prestadora,Estado,Instalacao,Manutencao,Informatica,Telefonia
0,24 Carrot Care,AC,5,0,1,5
1,Accel,AL,3,3,6,3
2,Aclima Group,AP,10,2,4,2
3,Acumentor,AM,3,9,0,0
4,Adrenalize,BA,9,6,7,7


In [2]:
## Insira aqui o dataframe para Precos

body = client_c86a1d2d7061465cb304eb925d845ec7.get_object(Bucket='workshoptim-donotdelete-pr-6u07kwfqd4dp4h',Key='Precos.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,Prestadora,Estado,AC,AL,AP,AM,BA,CE,DF,ES,...,PI,RJ,RN,RS,RO,RR,SC,SP,SE,TO
0,24 Carrot Care,AC,242,772,676,643,787,661,860,848,...,975,545,762,544,968,934,892,934,627,756
1,Accel,AL,670,615,797,874,698,648,958,843,...,503,711,791,989,696,920,808,605,762,726
2,Aclima Group,AP,814,988,687,935,713,767,645,896,...,984,571,709,797,931,534,740,756,538,687
3,Acumentor,AM,978,535,611,721,965,741,846,507,...,991,890,807,614,501,832,867,846,731,551
4,Adrenalize,BA,645,548,790,622,698,520,571,737,...,928,664,560,706,964,782,860,677,762,729


In [3]:
## Insira aqui o dataframe para OS

body = client_c86a1d2d7061465cb304eb925d845ec7.get_object(Bucket='workshoptim-donotdelete-pr-6u07kwfqd4dp4h',Key='OS.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()

,OS,UF,TipoServico,DiasAberto
0,OS9873085197,AP,Instalacao,2
1,OS5560490250,RJ,Informatica,1
2,OS4107464486,ES,Manutencao,2
3,OS4745956297,GO,Manutencao,7
4,OS0671550271,ES,Manutencao,2


Como pode ver, o IBM Watson Studio automaticamente cria o código para fazer a conexão com o dataset e trazê-lo. Porém, ele identifica os dataframes usando um nome padrão como *df_data_<numero>*. Vamos renomeá-los abaixo para algo mais legível:

In [4]:
df_tecnicos = df_data_1
df_precos = df_data_2
df_os = df_data_3

Pronto! Agora podemos começar a codificar o nosso modelo de otimização!

## Criando o modelo de otimização

Primeiramente, vamos preparar os dados para o modelo de otimização. Para isso, vamos pegar os dados que formarão nossas variáveis e deixá-los prontos. No caso, neste modelo, cada variável nossa é um valor binário na interseção entre OS, Prestador e Tipo de Serviço. Ou seja, teremos um cubo OS por Prestador por Tipo que receberá 1 se aquela OS estiver alocada para aquele prestador para aquele tipo de serviço ou 0 caso contrário.

O uso de um cubo nessa situação facilita quando formos calcular as restrições, como veremos mais a frente. Sendo assim, temos que preparar a lista de OS, Tipo de Serviço e Prestadora na célula abaixo, o que fazemos com Pandas:

In [5]:
# Usando Pandas, busque os valores únicos de Prestadora (do dataset Tecnicos), Tipos de Servico (do dataset OS) e OS (do dataset OS)

prestadoras = df_tecnicos['Prestadora'].unique()
tipos = df_os['TipoServico'].unique()
os = df_os['OS'].unique()

Para montar o modelo de otimização, necessitaremos da biblioteca *docplex* do IBM Decision Optimization. Caso a biblioteca não esteja instalada, podemos instala-lá utilizando o comando da célula abaixo:

In [6]:
!pip install docplex

Requirement not upgraded as not directly required: docplex in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: docloud>=1.0.315 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from docplex)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from docplex)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from docplex)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->docplex)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->docplex)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->docplex)
Requir

Uma vez que temos isso pronto, podemos agora começar a montagem do modelo. Para isso, vamos inicialmente importar as bibliotecas do IBM Decision Optimization e instanciar o nosso modelo na célula abaixo. Note que iremos aqui utilizar o serviço do DOCplexCloud, na nuvem, para executar o nosso modelo de otimização. Para isso, necessitaremos da URL e chave de API que colocamos em duas variáveis abaixo. Se quiséssemos executar localmente, necessitaríamos simplesmente colocar essas duas variáveis com o valor *None* e pronto, nenhuma outra alteração no código é necessária!

In [7]:
# Importando a biblioteca
from docplex.mp.model import Model, Context

# Instanciando o contexto, onde definimos se executaremos localmente ou na nuvem
url = 'https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/'
key = 'api_7f85a98f-5d2e-4a25-91b1-1ca54ef95be3'

context = Context.make_default_context()
context.solver.docloud.url = url
context.solver.docloud.key = key
context.solver.agent = 'docloud'

# Instanciando o modelo
# É importante darmos um nome para podermos identificar no otimizador qual job está em execução!
mdl = Model(name="otimizador_os",context=context)

Agora que já instanciamos o modelo, podemos começar a modelá-lo. Vamos começar definindo nosso cubo com as nossas variáveis. Em um modelo de otimização, as variáveis são as "alavancas" do nosso processo de planejamento, ou seja, aquelas decisões que estamos efetivamente querendo tomar. Nesse caso, queremos definir qual OS será alocada em qual capacidade para qual prestadora.

In [8]:
# Crie aqui o cubo de variáveis do modelo
cubo_vars = mdl.binary_var_cube(os,prestadoras,tipos)

Pronto! Agora podemos começar a trabalhar com as restrições. Em um modelo de otimização, as restrições são os limites que temos no nosso processo de tomada de decisão e que restringem nosso espaço de soluções. Por exemplo, no nosso caso em questão temos as seguintes restrições:

* Uma prestadora não pode ter mais alocações do que sua capacidade
* Uma OS só pode ser alocada a uma prestadora, em um tipo de serviço
* A OS só pode ser alocada no seu tipo de serviço

Vamos então criar estas restrições no nosso modelo!

In [9]:
# Crie a restrição de uma única prestadora e tipo de serviço para cada OS
# No cubo, isso significa que se somarmos todos os valores do cubo para uma determinada OS esse valor deve ser igual ou menor que 1
mdl.add_constraints(mdl.sum(cubo_vars[o,p,t] for p in prestadoras for t in tipos) <= 1 for o in os)

# Crie a restrição de que uma OS só pode ser alocada no seu tipo de serviço
# No cubo, isso significa que nos tipos de serviços diferentes de uma determinada OS, a soma nas prestadoras tem que ser 0
for t in tipos:
    for o in os:
        if df_os[df_os['OS'] == o]['TipoServico'].values[0] == t:
            mdl.add_constraint(mdl.sum(cubo_vars[o,p,t] for p in prestadoras) == 0)
            
# Crie a restrição de que uma prestadora não pode ter mais alocações do que sua capacidade
# No cubo, isso significa que para um determinado tipo e uma determinada prestadora, a soma nas OSs não pode ser maior que a capacidade no dataset
mdl.add_constraints(mdl.sum(cubo_vars[o,p,t] for o in os) <= df_tecnicos[df_tecnicos['Prestadora'] == p][t].values[0] for p in prestadoras for t in tipos)

[docplex.mp.LinearConstraint[](_x1+_x253+_x505+_x757+_x1009+_x1261+_x1513+_x1765+_x2017+_x2269+_x2521+_x2773+_x3025+_x3277+_x3529+_x3781+_x4033+_x4285+_x4537+_x4789+_x5041+_x5293+_x5545+_x5797+_x6049+_x6301+_x6553+_x6805+_x7057+_x7309+_x7561+_x7813+_x8065+_x8317+_x8569+_x8821+_x9073+_x9325+_x9577+_x9829+_x10081+_x10333+_x10585+_x10837+_x11089+_x11341+_x11593+_x11845+_x12097+_x12349+_x12601+_x12853+_x13105+_x13357+_x13609+_x13861+_x14113+_x14365+_x14617+_x14869+_x15121+_x15373+_x15625+_x15877+_x16129+_x16381+_x16633+_x16885+_x17137+_x17389+_x17641+_x17893+_x18145+_x18397+_x18649+_x18901+_x19153+_x19405+_x19657+_x19909+_x20161+_x20413+_x20665+_x20917+_x21169+_x21421+_x21673+_x21925+_x22177+_x22429+_x22681+_x22933+_x23185+_x23437+_x23689+_x23941+_x24193+_x24445+_x24697+_x24949+_x25201+_x25453+_x25705+_x25957+_x26209+_x26461+_x26713+_x26965+_x27217+_x27469+_x27721+_x27973+_x28225+_x28477+_x28729+_x28981+_x29233+_x29485+_x29737+_x29989+_x30241+_x30493+_x30745+_x30997+_x31249+_x31501+_x31753

Agora, vamos para a parte final que é definir o objetivo da nossa otimização. Nessa parte, vamos definir qual função queremos maximizar ou minimizar. No nosso caso atual, a intenção é fazer a alocação com o menor custo possível, e alocando o maior número de OS que pudemos atender. Sendo assim, temos que montar a função que calcula o nosso custo, calcula o número de OS não atendidas e então minimizá-la.

In [10]:
# Faça o cálculo do custo da solução
custo = 0
for o in os:
    # Pegar UF da OS para sabermos o preço cobrado pela prestadora
    UF_os = df_os[df_os['OS'] == o]['UF'].values[0]
    for p in prestadoras:
        # Pegar o preço cobrado pela prestadora para esta OS
        custo_os = df_precos[df_precos['Prestadora'] == p][UF_os].values[0]
        # Ver se a OS foi alocada e somar o custo
        custo += mdl.sum(cubo_vars[o,p,t] for t in tipos)*custo_os
        
# Agora, penalizamos o modelo pelas OS não atendidas somando ao custo um alto valor
nao_atendidas = len(os) - mdl.sum(cubo_vars[o,p,t] for o in os for p in prestadoras for t in tipos)
penalidade = nao_atendidas*1500
        
# Agora, criamos o objetivo de minimizar o custo e a penalidade
mdl.minimize(custo+penalidade)

# Para podermos comparar soluções, é interessante também inserir os KPIs que queremos medir
# Podemos então colocar o custo final e as OS não atendidas como KPIs
mdl.add_kpi(custo, publish_name="Custo")
mdl.add_kpi(nao_atendidas, publish_name="NaoAtendidas")

DecisionKPI(name=NaoAtendidas,expr=-_x1-_x2-_x3-_x4-_x5-_x6-_x7-_x8-_x9-_x10-_x11-_x12-_x13-_x14-_x..)

Agora, finalmente, vamos executar a otimização do problema e buscar o status da solução!

In [11]:
# Execute a otimização
mdl.solve()

# Imprima o resultado e os detalhes da solução
print(mdl.get_solve_status())
print(mdl.get_solve_details())

# Imprima os valores dos KPIs
print(mdl.report_kpis())

JobSolveStatus.OPTIMAL_SOLUTION
status  = integer optimal solution
time    = 4.69914 s.
problem = MILP
gap     = 0%

*  KPI: Custo        = 481094.000
*  KPI: NaoAtendidas = 0.000
None


E podemos imprimir também qual foi a solução do problema, ou seja, qual a alocação ótima de acordo com o nosso modelo.

In [12]:
# Imprima a solução do problema
print(mdl.solution)

solution for: otimizador_os
objective: 481094
_x40961=1
_x118853=1
_x202758=1
_x81928=1
_x187053=1
_x184331=1
_x96269=1
_x131089=1
_x77848=1
_x159769=1
_x114716=1
_x186397=1
_x165918=1
_x196640=1
_x131761=1
_x219171=1
_x104486=1
_x172071=1
_x122920=1
_x6186=1
_x86061=1
_x98350=1
_x32815=1
_x77872=1
_x145458=1
_x178229=1
_x249913=1
_x14394=1
_x194619=1
_x120892=1
_x61505=1
_x136545=1
_x190532=1
_x10309=1
_x12361=1
_x22604=1
_x108621=1
_x147534=1
_x138254=1
_x151633=1
_x137301=1
_x4184=1
_x247898=1
_x169811=1
_x20575=1
_x128358=1
_x59489=1
_x241762=1
_x112739=1
_x36966=1
_x88167=1
_x53354=1
_x235629=1
_x192623=1
_x168049=1
_x125044=1
_x9999=1
_x55414=1
_x157822=1
_x70042=1
_x228033=1
_x364=1
_x26758=1
_x102537=1
_x74776=1
_x143503=1
_x204945=1
_x16530=1
_x174227=1
_x153749=1
_x221335=1
_x120985=1
_x250565=1
_x162170=1
_x116892=1
_x73886=1
_x225439=1
_x205510=1
_x60444=1
_x51366=1
_x67751=1
_x34990=1
_x250031=1
_x202928=1
_x133297=1
_x146121=1
_x219318=1
_x176311=1
_x18616=1
_x30905=1
_x1

A impressão da solução aparece com os nomes internos das variáveis, mas podemos acessar os valores de cada alocação específica pela propriedade *solution_value* do cubo de variáveis.

In [13]:
# Imprimir qual a prestadora selecionada para cada OS
for o in os:
    for p in prestadoras:
        for t in tipos:
            if cubo_vars[(o,p,t)].solution_value != 0:
                print(o + ': ' + p + '(' + t + ')')

OS9873085197: LegCol(Informatica)
OS5560490250: Data Dot(Telefonia)
OS4107464486: Dot Compass(Telefonia)
OS4745956297: Isotronic(Instalacao)
OS0671550271: Dot Compass(Informatica)
OS3001818687: BaconGrease.com(Instalacao)
OS5944620831: Beyond Time(Telefonia)
OS8407094602: Beyond Time(Instalacao)
OS6620771686: Adrenalize(Telefonia)
OS5738215542: BaconGrease.com(Informatica)
OS9170640387: GenSink(Manutencao)
OS2288618876: Eargo(Manutencao)
OS2151676793: Bluegrain(Informatica)
OS4179056219: Isotronic(Informatica)
OS1905843082: Adrenalize(Instalacao)
OS9168314190: Comline(Informatica)
OS6663478375: Equicom(Telefonia)
OS0574980633: Adrenalize(Telefonia)
OS0602567047: LegCol(Telefonia)
OS1675068632: Geekus(Informatica)
OS3946603644: Bedding Down(Manutencao)
OS2925334369: Dot Compass(Instalacao)
OS1104600529: Data Dot(Instalacao)
OS5286046446: Eargo(Telefonia)
OS1837094623: Eargo(Informatica)
OS7823076259: Concert Communications(Telefonia)
OS5918854673: Flexigen(Instalacao)
OS6106316016: Call

OS6629806401: Flotonic(Informatica)
OS0769361669: Digitalus(Manutencao)
OS1958855914: Biohab(Informatica)
OS7076551966: Beyond Time(Instalacao)
OS7084752148: Callflex(Telefonia)
OS9519075428: Harmoney(Telefonia)
OS5194693367: Nexgene(Telefonia)
OS2189748013: DigiGate(Telefonia)
OS6684275288: Halo Systems(Telefonia)
OS7524158715: Beyond Time(Instalacao)
OS2554841949: Isotronic(Manutencao)
OS2314369651: Halo Systems(Telefonia)
OS4830188923: Dot Compass(Telefonia)
OS3995808563: Accel(Informatica)
OS7952896137: Concert Communications(Instalacao)
OS7844907174: BaconGrease.com(Informatica)
OS1790351469: Halo Systems(Manutencao)
OS4088639643: Manglo(Telefonia)
OS7267857863: 24 Carrot Care(Telefonia)
OS8527297023: Data Dot(Informatica)
OS6674153477: Eargo(Informatica)
OS9698843107: Acumentor(Manutencao)
OS1577356602: Dot Compass(Telefonia)
OS9362428951: Dot Compass(Informatica)
OS5557218814: Bluegrain(Instalacao)
OS2714616007: Networthservices(Instalacao)
OS7278146426: Omatom(Telefonia)
OS5639

**Pronto!** Nosso problema de otimização está resolvido!

## Exercício

Tente modificar o modelo acima para adicionar novas restrições ou objetivos! Por exemplo:

* Ordens de serviço abertas há mais de 5 dias devem obrigatoriamente ser atendidas
* Acrescentar os dias que uma ordem de serviço está aberta como penalidade no objetivo, de forma que ordens abertas há mais tempo tenham prioridade